# Testing

---

Aditya Marathe

## Imports

In [1]:
import os

import h5py

import pandas as pd

import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import dotenv

dotenv.load_dotenv();

In [3]:
import sys

sys.path.insert(1, './../')

In [4]:
from utils import *
from cuts import *

## Loading the data

In [5]:
data = h5py.File(os.environ[LocalDatasets.COPYMERGED_C9])

## Exploring the data

In [6]:
## Print all the keys (and their children) in my file

for i, key in enumerate(data.keys()):
    title = '(' + str(i + 1) + ') ' + key
    print(title)
    print('-' * len(title))
    for sub_key in data[key].keys():
        length = len(data[key][sub_key])
        print('\t' + sub_key + '\tLength: ' + str(length))
    
    print('\n')

(1) rec.energy.numu
-------------------
	E	Length: 141478
	angleE	Length: 141478
	angleerror	Length: 141478
	batch	Length: 141478
	calccE	Length: 141478
	cycle	Length: 141478
	evt	Length: 141478
	hadcalE	Length: 141478
	hadtrkE	Length: 141478
	lstmmuon	Length: 141478
	lstmmuon_opphorn	Length: 141478
	lstmnu	Length: 141478
	lstmnu_opphorn	Length: 141478
	ndhadcalactE	Length: 141478
	ndhadcalcatE	Length: 141478
	ndhadcaltranE	Length: 141478
	ndhadtrkactE	Length: 141478
	ndhadtrkcatE	Length: 141478
	ndhadtrktranE	Length: 141478
	ndtrkcalactE	Length: 141478
	ndtrkcalcatE	Length: 141478
	ndtrkcaltranE	Length: 141478
	ndtrklenact	Length: 141478
	ndtrklencat	Length: 141478
	ndtrktranx	Length: 141478
	ndtrktrany	Length: 141478
	recomuonE	Length: 141478
	recotrkcchadE	Length: 141478
	regcvnhadE	Length: 141478
	regcvnhadE_opphorn	Length: 141478
	run	Length: 141478
	shiftedtrkccE	Length: 141478
	subevt	Length: 141478
	subrun	Length: 141478
	trkccE	Length: 141478
	trknonqeE	Length: 141478
	trkqeE	

In [7]:
# col = 'rec.energy.numu'
col = 'rec.mc.nu'
for i in range(1_000):
    print(
        'Run ' + str(data[col]['run'][i][0]) + ' '
        + 'Sub-run ' + str(data[col]['subrun'][i][0]) + ' '
        + 'Event ' + str(data[col]['evt'][i][0]) + ' '
        + 'Sub-event ' + str(data[col]['subevt'][i][0]) + ' '
        + 'Cycle ' + str(data[col]['cycle'][i][0])
    )

Run 14850 Sub-run 5 Event 2 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 3 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 5 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 7 Sub-event 3 Cycle 0
Run 14850 Sub-run 5 Event 9 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 11 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 13 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 15 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 16 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 17 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 20 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 22 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 25 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 26 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 28 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 31 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 32 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 33 Sub-event 3 Cycle 0


Run 14850 Sub-run 5 Event 38 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 39 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 40 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 42 Sub-event 3 Cycle 0
Run 14850 Sub-run 5 Event 43 Sub-event 4 Cycle 0
Run 14850 Sub-run 5 Event 47 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 48 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 51 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 52 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 56 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 57 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 63 Sub-event 2 Cycle 0
Run 14850 Sub-run 5 Event 64 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 66 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 68 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 69 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 76 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 77 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 78 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 Event 79 Sub-event 1 Cycle 0
Run 14850 Sub-run 5 

In [8]:
info_string = """\
Run range {}-{}
Sub-run range {}-{}
Sub-run mean {}

Event range {}-{}
Sub-event range {}-{}
Sub-event mean {}

Cycle range {}-{}
"""

for i, key in enumerate(data.keys()):
    title = '(' + str(i + 1) + ') ' + key
    print(title)
    print('-' * len(title))
    
    try:
        print(
            info_string.format(
                np.min(data[key]['run']),
                np.max(data[key]['run']),
                np.min(data[key]['subrun']),
                np.max(data[key]['subrun']),
                np.mean(data[key]['subrun']),

                np.min(data[key]['evt']),
                np.max(data[key]['evt']),
                np.min(data[key]['subevt']),
                np.max(data[key]['subevt']),
                np.mean(data[key]['subevt']),

                np.min(data[key]['cycle']),
                np.max(data[key]['cycle'])
            )
        )

    except Exception as e:
        print(e)
    
    print('\n')

(1) rec.energy.numu
-------------------
Run range 14850-15021
Sub-run range 0-63
Sub-run mean 30.553054185102983

Event range 1-1000
Sub-event range 1-11
Sub-event mean 1.6384243486619827

Cycle range 0-1



(2) rec.energy.numu.hadclust
----------------------------
Run range 14850-15021
Sub-run range 0-63
Sub-run mean 30.553054185102983

Event range 1-1000
Sub-event range 1-11
Sub-event mean 1.6384243486619827

Cycle range 0-1



(3) rec.mc
----------
Run range 14850-15021
Sub-run range 0-63
Sub-run mean 30.553054185102983

Event range 1-1000
Sub-event range 1-11
Sub-event mean 1.6384243486619827

Cycle range 0-1



(4) rec.mc.nu
-------------
Run range 14850-15021
Sub-run range 0-63
Sub-run mean 30.52465823431711

Event range 1-1000
Sub-event range 1-7
Sub-event mean 1.3787104467441762

Cycle range 0-1



(5) rec.mc.nu.beam
------------------
Run range 14850-15021
Sub-run range 0-63
Sub-run mean 30.52465823431711

Event range 1-1000
Sub-event range 1-7
Sub-event mean 1.378710446744176

In [9]:
info_string = """\
Unique Runs {}
Unique Sub-runs {}

Unique Events {}
Unique Sub-events {}

Unique Cycle {}
"""

for i, key in enumerate(data.keys()):
    title = '(' + str(i + 1) + ') ' + key
    print(title)
    print('-' * len(title))

    try:
        print(info_string.format(
                np.unique(data[key]['run']),
                np.unique(data[key]['subrun']),
                np.unique(data[key]['evt']),
                np.unique(data[key]['subevt']),
                np.unique(data[key]['cycle'])
            )
        )
    except Exception as e:
        print(e)

    print('\n')

(1) rec.energy.numu
-------------------
Unique Runs [14850 14852 14853 14854 14859 14867 14869 14870 14871 14873 14874 14875
 14881 14886 14887 14889 14890 14891 14994 14995 14996 14997 14998 14999
 15000 15001 15003 15004 15005 15006 15007 15008 15009 15010 15011 15012
 15013 15014 15019 15021]
Unique Sub-runs [ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 25 26 27
 28 29 30 31 32 35 36 37 38 39 40 42 44 45 46 50 51 53 54 55 56 57 58 59
 60 61 63]

Unique Events [   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   95   96   97   98
   99  100  101  1

## Creating a `DataFrame`

In [10]:
def get_column(
        parent_key: str,
        child_key: str
    ):
    return data[parent_key][child_key][0:-1].flatten()

In [11]:
def get_value(
        parent_key: str, 
        child_key: str,
        run: int,
        sub_run: int,
        evt: int,
        sub_evt: int,
        cycle: int
    ):
    value_array = get_column(parent_key, child_key)
    
    bool_locator = (
        (get_column(parent_key, 'run') == run)
        & (get_column(parent_key, 'subrun') == sub_run)
        & (get_column(parent_key, 'evt') == evt)
        & (get_column(parent_key, 'subevt') == sub_evt)
        & (get_column(parent_key, 'cycle') == cycle)
    )

    return value_array[bool_locator]

In [43]:
## Variables that we will extract
# Source: Domain Generalisation with ML in the NOvA experiment by Andrew Thomas
#         Cleve Sutton (University of North Carolina at Charlotte)

useful_vars = {
    
    # Energies ----------------------------------------------------------------

    'rec.energy.numu': [
        'E',  # Old predictor
        'calccE',
        'hadcalE',
        'hadtrkE',
        'lstmmuon',  # Most recent LSTM predictors
        'lstmnu',
        'regcvnhadE',
        'trkccE',
        'recomuonE'  # --> Reconstructed energy...
    ],
    'rec.energy.numu.hadclust': [
        'calE',
        'nhit'
    ],

    # Selection stuff ---------------------------------------------------------

    'rec.sel.contain': [
        'kalfwdcell',
        'kalbakcell',
        'cosfwdcell',
        'cosbakcell'
    ],
    'rec.sel.cosrej': [
        # For FHC (Forward Horn Current)
        'numucontpid2020fhc'
    ],
    'rec.sel.cvnloosepreselptp': [
        'cosmicid',
        'ncid',
        'nueid',
        'numuid',
        'nutauid'
    ],
    'rec.sel.remid': [
        'pid'
    ],
    'rec.sel.scann': [
        'scpid'
    ],
    'rec.sel.veto': [
        'keep'
    ],

    # Slice stuff -------------------------------------------------------------

    'rec.slc': [
        'calE',  # Calorimetric energy of all hits in this slice
        'firstplane',
        'lastplane',
        'nhit',
        'ncontplanes'
    ],

    # Spill info --------------------------------------------------------------

    'rec.spill.cosmiccvn': [
        'passSel'
    ],

    # Track info --------------------------------------------------------------

    'rec.trk.cosmic': [
        'ntracks'
    ],
    'rec.trk.kalman': [
        'idxremid',
        'ntracks'
    ],
    'rec.trk.kalman.tracks': [
        'dir.x',  # --> Needed for calculating kinematics
        'dir.y',
        'dir.z',
        'nhit',
        'nhitx',
        'nhity',
        'calE',  # Calorimetric energy of all hits in the prong
        'nplane',
        'maxplanecont'
    ],

    # MC Truth ----------------------------------------------------------------

    'rec.mc': [
        'nnu'
    ],
    'rec.mc.nu': [
        'pdg',
        'mode',
        'iscc'
    ],
    'rec.mc.nu.beam': [
        'potnum'  # Number of POT (Protons On Target)
    ]
}

In [20]:
require_manual_fill = [
    'rec.mc.nu',
    'rec.mc.nu.beam',
    'rec.training.cvnmaps',
    'rec.training.trainingdata',
    'rec.trk.kalman.tracks'
]

In [23]:
# For future reference:
# https://stackoverflow.com/questions/47531584/numpy-where-as-range-indices
# https://stackoverflow.com/questions/40468069/merge-two-dataframes-by-index

In [52]:
def get_columns_special(parent_key: str) -> pd.DataFrame:
    """\
    
    """
    run_column = get_column(parent_key, 'run')
    subrun_column = get_column(parent_key, 'subrun')
    evt_column = get_column(parent_key, 'evt')
    subevt_column = get_column(parent_key, 'subevt')
    cycle_column = get_column(parent_key, 'cycle')

    zipped_index_array_new = np.column_stack(
        (
            run_column,
            subrun_column,
            evt_column,
            subevt_column,
            cycle_column
        )
    )

    zipped_index_array_new, index_array = np.unique(
        zipped_index_array_new, 
        axis=0, 
        return_inverse=True
    )

    multi_index_new = pd.MultiIndex.from_tuples(
        zipped_index_array_new.tolist(), 
        names=[
            'run',
            'subrun',
            'evt',
            'subevt',
            'cycle'
        ]
    )

    df_new = pd.DataFrame(index=multi_index_new)

    for child_key in useful_vars[parent_key]:
        column_values = get_column(parent_key, child_key)
        split_indices = np.where(index_array[:-1] != index_array[1:])[0] + 1
        grouped_values = np.split(column_values, split_indices)
        df_new[parent_key + '.' + child_key] = grouped_values

    return df_new

In [53]:
multi_index = pd.MultiIndex.from_tuples(
    np.column_stack((
        get_column('rec.energy.numu', 'run'),
        get_column('rec.energy.numu', 'subrun'),
        get_column('rec.energy.numu', 'evt'),
        get_column('rec.energy.numu', 'subevt'),
        get_column('rec.energy.numu', 'cycle')
    )).tolist(), 
    names=[
        'run',
        'subrun',
        'evt',
        'subevt',
        'cycle'
    ]
)

df = pd.DataFrame(index=multi_index)

In [54]:
for parent_key in useful_vars:
    if parent_key in require_manual_fill:
        print('Skipped ' + parent_key)
        continue
    
    for child_key in useful_vars[parent_key]:
        df[parent_key + '.' + child_key] = get_column(parent_key, child_key)

Skipped rec.trk.kalman.tracks
Skipped rec.mc.nu
Skipped rec.mc.nu.beam


In [55]:
df_kalman = get_columns_special('rec.trk.kalman.tracks')
df_nu = get_columns_special('rec.mc.nu')
df_nu_beam = get_columns_special('rec.mc.nu.beam')

In [56]:
df = pd.merge(df, df_kalman, left_index=True, right_index=True, how='outer')
df = pd.merge(df, df_nu, left_index=True, right_index=True, how='outer')
df = pd.merge(df, df_nu_beam, left_index=True, right_index=True, how='outer')

In [58]:
# Reorder the columns...
column_order = list()

for p_key in useful_vars:
    for c_key in useful_vars[p_key]:
        column_order.append(p_key + '.' + c_key)

df = df[column_order]

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 141478 entries, (14850, 5, 2, 1, 0) to (15021, 2, 1000, 2, 0)
Data columns (total 47 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   rec.energy.numu.E                   141477 non-null  float32
 1   rec.energy.numu.calccE              141477 non-null  float32
 2   rec.energy.numu.hadcalE             141477 non-null  float32
 3   rec.energy.numu.hadtrkE             141477 non-null  float32
 4   rec.energy.numu.lstmmuon            141477 non-null  float32
 5   rec.energy.numu.lstmnu              141477 non-null  float32
 6   rec.energy.numu.regcvnhadE          141477 non-null  float32
 7   rec.energy.numu.trkccE              141477 non-null  float32
 8   rec.energy.numu.recomuonE           141477 non-null  float32
 9   rec.energy.numu.hadclust.calE       141477 non-null  float32
 10  rec.energy.numu.hadclust.nhit       141477 non-null  float

In [68]:
# Can I do this? --> Why is there less values for MC and Kalman tracks?
# TODO: Check this...
df_no_nans = df.dropna()

## Track kinematics

In [25]:
# Calculate the track kinematics
# The code below is directly copied from the `pidFunWithScann.ipynb` module

# Idea: The data from the FD is in a different coordinate system to the data 
#       used for the NuMI simulation. Hence, we need to make some convertations
#       before we can start calculating the track kinematics.

# Question: How were these coordinate systems implemented?

In [26]:
## FD Transform

# r_offset vector
r_off_fd = np.array(
    [57.666820, -51777.408547, -808852.640542]
)

# rotationa matrix
m_rot_fd = np.array(
    [
        [0.99990622174    , 0.000472822894411, 0.0136866417481],
        [0.000472822894411, 0.997616062703   , -0.0690070132296],
        [-0.0136866417481 , 0.0690070132296  , 0.997522284444]
    ]
)

m_rot_fd_inv = np.linalg.inv(m_rot_fd)

## ND Transform

# r_offset vector
r_off_nd=np.array(
    [2.269447, 61.001882, -991.131313]
) 

# rotation matrix
m_rot_nd=np.array(
    [
        [0.99990   , 3.0533E-6 , 1.4112E-2],
        [8.2300E-4 , 0.99831   , 5.8097E-2],
        [-1.4088E-2, -5.8103E-2, 0.99821]
    ]
)

m_rot_nd_inv = np.linalg.inv(m_rot_nd)

# Block centres from the survey measurements in e.g. DocDb-13501

# Question: What is any of this? 
#           Where can I find out more about these calculations?

fdBlkCtr = np.array([11037.705436,-4164.619757,810452.13544])
ndBlkCtr = np.array([11.793636,-2.928869,999.282492])

pointS = np.array([0,0,170.399278])

mczero = np.array([0,0,0])

point100 = np.array([0,0,100])
point600 = np.array([0,0,600])
point800 = np.array([0,0,800])


def convFDtoNuMI(r_fd):
    return np.matmul(m_rot_fd,r_fd-r_off_fd)


def convNuMItoFD(r_numi):
    return np.matmul(m_rot_fd_inv,r_numi)+r_off_fd


def convNDtoNuMI(r_nd):
    return np.matmul(m_rot_nd,r_nd-r_off_nd)


def convNuMItoND(r_numi):
    return np.matmul(m_rot_nd_inv,r_numi)+r_off_nd


r_fd=np.array([0,0,60])
print(convFDtoNuMI(r_fd))


def getDirAtDetectorFromPoint(whichDet,point):
    if whichDet == "FD":
        fdCentFromPoint = fdBlkCtr - point
        fdCentDist = np.linalg.norm(fdCentFromPoint)
        fdCfP = fdCentFromPoint / np.linalg.norm(fdCentFromPoint)
        r_fd_beamFront = convNuMItoFD(pointS + (fdCentDist - 29.942) * fdCfP)
        r_fd_beamBack = convNuMItoFD(pointS + (fdCentDist + 29.942) * fdCfP)
        r_fd_beamDir = r_fd_beamBack - r_fd_beamFront

        return r_fd_beamDir
    
    elif whichDet == "ND":
        ndCentFromPoint=ndBlkCtr-point
        ndCentDist=np.linalg.norm(ndCentFromPoint)
        ndCfP=ndCentFromPoint/np.linalg.norm(ndCentFromPoint)
        r_nd_beamFront=convNuMItoND(pointS+(ndCentDist - 29.942) * ndCfP)
        r_nd_beamBack=convNuMItoND(pointS+(ndCentDist + 29.942) * ndCfP)
        r_nd_beamDir=r_nd_beamBack-r_nd_beamFront
        
        return r_nd_beamDir
    
    else:
        print("Don't have detector", whichDet)
        
        
r_fd_beamDir = getDirAtDetectorFromPoint("FD", pointS)
r_fd_beamDirUnit = r_fd_beamDir / np.linalg.norm(r_fd_beamDir)
print("r_fd_beamDir", r_fd_beamDir)
print("r_fd_beamDirUnit", r_fd_beamDirUnit)
r_nd_beamDir = getDirAtDetectorFromPoint("ND", pointS)
r_nd_beamDirUnit = r_nd_beamDir / np.linalg.norm(r_nd_beamDir)

for point in [mczero,point100,pointS,point600,point800]:
    print(point)
    rp = getDirAtDetectorFromPoint("ND", point)
    rp /= np.linalg.norm(rp)
    print("\t", rp, 180 * np.arctan(rp[0] / rp[2]) / np.pi, 180 * np.arctan(rp[1] / rp[2]) / np.pi)

print("r_nd_beamDir", r_nd_beamDir, 180 * np.arctan(r_nd_beamDir[1] / r_nd_beamDir[2]) / np.pi)
print("r_nd_beamDirUnit", r_nd_beamDirUnit)


[ 11038.11764868  -4166.69810204 810482.17869079]
r_fd_beamDir [-4.08908094e-03  3.82534312e+00  5.97616950e+01]
r_fd_beamDirUnit [-6.82833636e-05  6.38792185e-02  9.97957635e-01]
[0 0 0]
	 [-0.00228449 -0.06102058  0.99813389] -0.13113611583944212 -3.498404261662031
[  0   0 100]
	 [-9.72040557e-04 -6.13470174e-02  9.98116025e-01] -0.055798927645348176 -3.5171352997696474
[  0.         0.       170.399278]
	 [ 1.41822179e-04 -6.16239720e-02  9.98099427e-01] 0.008141285333660259 -3.5330320844841254
[  0   0 600]
	 [ 0.01544502 -0.0654208   0.99773823] 0.8868699638910319 -3.751462511134568
[  0   0 800]
	 [ 0.04501506 -0.07271388  0.99633646] 2.5868974399857207 -4.1741174416391935
r_nd_beamDir [ 8.49288940e-03 -3.69029431e+00  5.97702568e+01] -3.5330320844841254
r_nd_beamDirUnit [ 1.41822179e-04 -6.16239720e-02  9.98099427e-01]


In [27]:
# Calculating the track kinematics
# Again, this code is adapted from the `pidFunWithScann.ipynb` module

In [28]:
# Proton mass (GeV^-1)
M_p = 0.938272

In [29]:
def fillTrackKinematics(td: dict) -> None:
    trackIndices = np.cumsum(td['rec.trk.kalman.ntracks'])
    trackIndices = np.concatenate(([0],trackIndices)).astype(int)

    # td['trackIndices'] = trackIndices

    # Cos(Theta) of the angle between the muon track and original track

    # Calculation:
    # (Kalman track direction vector) . (FD beam direction vector) = | // | | // | cos(theta)
    # cos(theta) = ( // . // ) / (| // | | // |) = ( // . // ) since they are unit vectors... 

    # td['calc.cosBeam'] 
    cos_beam = td['rec.trk.kalman.tracks.dir.x'] * r_fd_beamDirUnit[0] + td['rec.trk.kalman.tracks.dir.y'] * r_fd_beamDirUnit[1] + td['rec.trk.kalman.tracks.dir.z'] * r_fd_beamDirUnit[2]
    td['der.cosThetaFirst'] = cos_beam * [trackIndices[0:-1]] # Magic line to select only those cos values that are from the first track
    
    # Question: Why do we need to get only the first track?

    # Momentum stuff
    
    # Tranverse Momentum / Muon Momentum
    td['der.ptpmuRatio'] = np.sqrt(1-td['cosBeamFirst'] * td['cosBeamFirst'])
    
    # Muon Momentum
    td['der.pmu']=np.sqrt(td['rec.energy.numu.lstmmuon']*td['rec.energy.numu.lstmmuon'] - (105.658e-3)**2)
    
    # Transverse Momentum
    td['der.pt'] = td['der.ptpmuRatio'] * td['der.pmu']

    # Q^2 and W
    td['recoQ2']= 2 * td['rec.energy.numu.lstmnu'] * (td['rec.energy.numu.lstmmuon'] - td['pmu'] * td['cosBeamFirst']) - (105.658e-3)**2
    td['recoW'] = np.sqrt(M_p * M_p + 2 * M_p * (td['rec.energy.numu.lstmnu'] - td['rec.energy.numu.lstmmuon']) - td['recoq2'])

# fillTrackKinematics(td=df)